In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [0]:
data = pd.read_csv("/content/drive/My Drive/vmware/Project/214551_better_timetable.csv")

In [0]:
data

,Unnamed: 0,Unnamed: 0.1,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time
0,10218,10218,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4
1,10219,10219,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4
2,10220,10220,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,4
3,10221,10221,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4
4,10222,10222,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449161,4639073,4639073,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,9
449162,4639074,4639074,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10
449163,4639075,4639075,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,13
449164,4639076,4639076,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10


In [0]:
data = data.drop(columns=data.columns[0])

In [0]:
data["query_time"] = pd.to_datetime(data.query_time)
data["query_time"] = [t.time() for t in data.query_time]

In [0]:
data["expected_arrival"] = pd.to_datetime(data.expected_arrival)
data["expected_arrival"] = [t.time() for t in data.expected_arrival]

In [0]:
data["date"] = pd.to_datetime(data.date)
data["date"] = [t.date() for t in data.date]

In [0]:
data["timetable"] = pd.to_datetime(data.timetable)
data["timetable"] = [t.time() for t in data.timetable]

In [0]:
data = data.drop_duplicates(["date", "query_time", "timetable"])

In [0]:
data

,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time
0,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4
1,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4
2,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,4
3,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4
4,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449161,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,9
449162,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10
449163,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,13
449164,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10


In [0]:
arrival = data[["date", "query_time", "timetable", "expected_arrival", "wait_time"]].copy()
arrival = arrival.sort_values(["date", "timetable", "query_time"], ascending=[True, True, True])

In [26]:
arrival

,date,query_time,timetable,expected_arrival,wait_time
0,2016-06-17,06:29:21,06:42:00,06:46:00,4
3,2016-06-17,06:32:25,06:42:00,06:46:00,4
6,2016-06-17,06:35:30,06:42:00,06:46:37,4
9,2016-06-17,06:38:36,06:42:00,06:46:37,4
13,2016-06-17,06:41:40,06:42:00,06:42:57,0
...,...,...,...,...,...
449161,2017-09-24,23:21:26,23:24:00,23:33:33,9
449162,2017-09-24,23:24:29,23:24:00,23:34:32,10
449163,2017-09-24,23:27:32,23:24:00,23:37:21,13
449164,2017-09-24,23:30:34,23:24:00,23:34:52,10


In [0]:
arrival = arrival.drop(columns=["query_time"])

In [0]:
arrival = arrival.drop_duplicates(["date", "timetable"])

In [29]:
arrival

,date,timetable,expected_arrival,wait_time
0,2016-06-17,06:42:00,06:46:00,4
1,2016-06-17,07:02:00,07:06:00,4
2,2016-06-17,07:22:00,07:26:00,4
61,2016-06-17,07:32:00,07:27:06,-5
12,2016-06-17,07:38:00,07:36:00,-2
...,...,...,...,...
449051,2017-09-24,22:04:00,22:14:00,10
449071,2017-09-24,22:24:00,22:34:00,10
449089,2017-09-24,22:44:00,22:54:00,10
449110,2017-09-24,23:04:00,23:14:00,10


In [30]:
data

,Unnamed: 0.1,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time
0,10218,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4
1,10219,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4
2,10220,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,4
3,10221,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4
4,10222,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449161,4639073,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,9
449162,4639074,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10
449163,4639075,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,13
449164,4639076,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10


Adding "true" arrival/wait times to each row

In [0]:
data["arrival"] = None
data["arrival_delay"] = None
data["wait_time"] = None

In [32]:
data

,Unnamed: 0.1,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time,arrival,arrival_delay
0,10218,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,None,None,None
1,10219,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,None,None,None
2,10220,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,None,None,None
3,10221,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,None,None,None
4,10222,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449161,4639073,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,None,None,None
449162,4639074,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,None,None,None
449163,4639075,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,None,None,None
449164,4639076,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,None,None,None


In [0]:
for date in data.date.unique():
    by_date = data[data.date == date]
    for time in by_date.timetable.unique():
        by_time = by_date[by_date.timetable == time]
        timetabled = arrival[arrival.date ==  date]
        timetabled = timetabled[timetabled.timetable == time]
        arrival_now = timetabled.at[timetabled.index[0], "expected_arrival"]
        wait_time = timetabled.at[timetabled.index[0], "wait_time"]

        for index in by_time.index:
            query = data.at[index, "query_time"]
            data.at[index, "arrival"] = arrival_now
            data.at[index, "arrival_delay"] = wait_time
            data.at[index, "wait_time"] = (arrival_now.hour*60 + arrival_now.minute) - (query.hour*60 + query.minute)

In [34]:
data

,Unnamed: 0.1,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time,arrival,arrival_delay
0,10218,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,17,06:46:00,4
1,10219,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,37,07:06:00,4
2,10220,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,57,07:26:00,4
3,10221,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,14,06:46:00,4
4,10222,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,34,07:06:00,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449161,4639073,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,13,23:34:00,10
449162,4639074,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10,23:34:00,10
449163,4639075,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,7,23:34:00,10
449164,4639076,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,4,23:34:00,10


In [35]:
timetabled

,date,timetable,expected_arrival,wait_time
449130,2017-09-24,23:24:00,23:34:00,10


Adding the wait time from the bus previous

In [0]:
data["delay_last"] = None
data["wait_time_last"] = None

In [37]:
5 // 2

2

In [49]:
for date in data.date.unique():
    by_date = data[data.date == date]
    for j in range(len(by_date.timetable.unique())):
        time = list(by_date.timetable.unique())[j]
        old_time = list(by_date.timetable.unique())[j]
        if j > 0:
            old_time = list(by_date.timetable.unique())[j-1]
        by_time = by_date[by_date.timetable == time]
        wait_time_last = data[data.date == date]
        wait_time_last = wait_time_last[wait_time_last.timetable == old_time]

        timetable_last = wait_time_last.at[wait_time_last.index[0], "timetable"]
        arrival_last = wait_time_last.at[wait_time_last.index[0], "expected_arrival"]
        wait_time = wait_time_last.at[wait_time_last.index[0], "wait_time"]

        for index in by_time.index:
            query = data.at[index, "query_time"]
            arr = data.at[index, "arrival"]
            tim = data.at[index, "timetable"]

            difference = (tim.hour*60 + tim.minute) - (timetable_last.hour*60 + timetable_last.minute)

            a = timedelta(hours=(difference // 60), minutes=(difference % 60))

            scaled_down = (datetime.combine(datetime(1,1,1),arrival_last) + timedelta(hours=(difference // 60), minutes=(difference % 60))).time()

            data.at[index, "delay_last"] = wait_time

            data.at[index, "wait_time_last"] = (scaled_down.hour*60 + scaled_down.minute) - (query.hour*60 + query.minute)


data

,Unnamed: 0.1,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time,arrival,arrival_delay,delay_last,wait_time_last
0,10218,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,17,06:46:00,4,17.0,17.0
1,10219,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,37,07:06:00,4,17.0,37.0
2,10220,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,57,07:26:00,4,37.0,57.0
3,10221,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,14,06:46:00,4,17.0,14.0
4,10222,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,34,07:06:00,4,17.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449161,4639073,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,13,23:34:00,10,57.0,13.0
449162,4639074,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10,23:34:00,10,57.0,10.0
449163,4639075,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,7,23:34:00,10,57.0,7.0
449164,4639076,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,4,23:34:00,10,57.0,4.0


In [0]:
data

Arrival from the preceding day

In [0]:
arrival = data[["date", "timetable", "arrival", "wait_time"]].copy()

In [0]:
arrival = arrival.drop_duplicates(["date", "timetable"])

In [0]:
data["arrival_-24"] = None
data["delay_-24"] = None
data["wait_time_-24"] = None

In [0]:
bank_holidays = ["2016-08-01", "2016-10-31", "2016-12-26", "2017-01-01", "2017-03-17", "2017-05-01", "2017-06-05",
                 "2017-04-17", "2017-08-07"]

b_h_objects = []
for da in bank_holidays:
    b_h_objects += [pd.to_datetime(da).date()]

In [0]:
for date in arrival[arrival.date >= date2].date.unique():
    by_date = arrival[arrival.date == date]
    for time in by_date.timetable.unique():
        by_time = by_date[by_date.timetable == time]
        new_day = date

        weekday = date.weekday()

        if new_day > arrival.date.unique()[0]:
            if weekday < 5:
                new_day = new_day - timedelta(days=1)
            else:
                new_day = new_day - timedelta(weeks=1)
        while new_day > arrival.date.unique()[0]:
            if new_day in list(arrival.date.unique()):
                if new_day.weekday() < 5 and date.weekday() < 5:
                    break
                if new_day.weekday() == 5 and date.weekday() == 5:
                    break
                if new_day.weekday() == 6 and date.weekday() == 6:
                    break
            if date.weekday() < 5:
                new_day = new_day - timedelta(days=1)
            else:
                new_day = new_day - timedelta(weeks=1)
            
        
        if new_day not in list(arrival.date.unique()):
            new_day = date
        
        applied = data[data.date == date]
        applied = applied[applied.timetable == time]

        yesterday = arrival[arrival.date == new_day]
        yesterday = yesterday[yesterday.timetable == time]

        

        for index in applied.index:

            query = data.at[index, "query_time"]

            try:
                arrival_24 = yesterday.at[yesterday.index[0], "arrival"]
                wait_time = yesterday.at[yesterday.index[0], "wait_time"]
            except IndexError:
                arrival_24 = data.at[index, "arrival"]
                wait_time = data.at[index, "wait_time"]


            data.at[index, "arrival_-24"] = arrival_24
            data.at[index, "delay_-24"] = wait_time
            data.at[index, "wait_time_-24"] = (arrival_24.hour*60 + arrival_24.minute) - (query.hour*60 + query.minute)
         
         


        

In [0]:
date2 = date

In [0]:
date

datetime.date(2016, 6, 30)

Add data for bus 48 hours previous

In [0]:
data["arrival_-48"] = None
data["delay_-48"] = None
data["wait_time_-48"] = None

In [0]:
for date in arrival.date.unique():
    by_date = arrival[arrival.date == date]
    for time in by_date.timetable.unique():
        by_time = by_date[by_date.timetable == time]
        new_day = date

        weekday = date.weekday()

        if new_day > arrival.date.unique()[0]:
            if weekday < 5:
                new_day = new_day - timedelta(days=2)
            else:
                new_day = new_day - timedelta(weeks=2)
        while new_day > arrival.date.unique()[0]:
            if new_day in list(arrival.date.unique()):
                if new_day.weekday() < 5 and date.weekday() < 5:
                    break
                if new_day.weekday() == 5 and date.weekday() == 5:
                    break
                if new_day.weekday() == 6 and date.weekday() == 6:
                    break
            if date.weekday() < 5:
                new_day = new_day - timedelta(days=1)
            else:
                new_day = new_day - timedelta(weeks=1)
            
        
        if new_day not in list(arrival.date.unique()):
            new_day = date
        
        applied = data[data.date == date]
        applied = applied[applied.timetable == time]

        yesterday = arrival[arrival.date == new_day]
        yesterday = yesterday[yesterday.timetable == time]

        

        for index in applied.index:

            query = data.at[index, "query_time"]

            try:
                arrival_24 = yesterday.at[yesterday.index[0], "arrival"]
                wait_time = yesterday.at[yesterday.index[0], "wait_time"]
            except IndexError:
                arrival_24 = data.at[index, "arrival"]
                wait_time = data.at[index, "wait_time"]


            data.at[index, "arrival_-48"] = arrival_24
            data.at[index, "delay_-48"] = wait_time
            data.at[index, "wait_time_-48"] = (arrival_24.hour*60 + arrival_24.minute) - (query.hour*60 + query.minute)
         
         


        

Code for arrivals a week prior

In [0]:
data["arrival_-week"] = None
data["wait_time_-week"] = None
data["delay_-week"] = None

In [0]:
for date in arrival.date.unique():
    by_date = arrival[arrival.date == date]
    for time in by_date.timetable.unique():
        by_time = by_date[by_date.timetable == time]
        new_day = date

        weekday = date.weekday()

        if new_day > arrival.date.unique()[0]:
            if weekday < 5:
                new_day = new_day - timedelta(weeks=1)
            else:
                new_day = new_day - timedelta(weeks=3)
        while new_day > arrival.date.unique()[0]:
            if new_day in list(arrival.date.unique()):
                if new_day.weekday() < 5 and date.weekday() < 5:
                    break
                if new_day.weekday() == 5 and date.weekday() == 5:
                    break
                if new_day.weekday() == 6 and date.weekday() == 6:
                    break
            if date.weekday() < 5:
                new_day = new_day - timedelta(days=1)
            else:
                new_day = new_day - timedelta(weeks=1)
            
        
        if new_day not in list(arrival.date.unique()):
            new_day = date
        
        applied = data[data.date == date]
        applied = applied[applied.timetable == time]

        yesterday = arrival[arrival.date == new_day]
        yesterday = yesterday[yesterday.timetable == time]

        

        for index in applied.index:

            query = data.at[index, "query_time"]

            try:
                arrival_24 = yesterday.at[yesterday.index[0], "arrival"]
                wait_time = yesterday.at[yesterday.index[0], "wait_time"]
            except IndexError:
                arrival_24 = data.at[index, "arrival"]
                wait_time = data.at[index, "wait_time"]


            data.at[index, "arrival_-week"] = arrival_24
            data.at[index, "delay_-week"] = wait_time
            data.at[index, "wait_time_-week"] = (arrival_24.hour*60 + arrival_24.minute) - (query.hour*60 + query.minute)

In [0]:
data = data[data.date == datetime(2019, 9, 24)]

In [0]:
data

,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time,arrival,arrival_wait,wait_time_last,arrival_-24,wait_time_-24,arrival_-48,wait_time_-48,arrival_-week,wait_time_-week


In [0]:
data[data.wait_time < -300].date.unique()

array([datetime.date(2016, 8, 1), datetime.date(2016, 10, 31),
       datetime.date(2017, 1, 8), datetime.date(2017, 4, 14),
       datetime.date(2017, 4, 15), datetime.date(2017, 6, 5),
       datetime.date(2017, 7, 27), datetime.date(2017, 7, 28),
       datetime.date(2017, 7, 30), datetime.date(2017, 7, 31),
       datetime.date(2017, 8, 2), datetime.date(2017, 8, 7)], dtype=object)

In [0]:
data.to_csv('/content/drive/My Drive/vmware/Project/214551_finalised_dataset.csv')

In [0]:
data2 = pd.read_csv("/content/drive/My Drive/vmware/Project/214551_finalised_dataset.csv")

In [10]:
data2

,Unnamed: 0,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time,arrival,arrival_wait,wait_time_last,arrival_-24,wait_time_-24,arrival_-48,wait_time_-48,arrival_-week,wait_time_-week
0,0,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4,06:46:00,4,4,06:46:00,4,06:46:00,4.0,06:46:00,4.0
1,1,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4,07:06:00,4,4,07:06:00,4,07:06:00,4.0,07:06:00,4.0
2,2,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,4,07:26:00,4,4,07:26:00,4,07:26:00,4.0,07:26:00,4.0
3,3,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4,06:46:00,4,4,06:46:00,4,06:46:00,4.0,06:46:00,4.0
4,4,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4,07:06:00,4,4,07:06:00,4,07:06:00,4.0,07:06:00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373014,449161,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,9,23:34:00,10,10,23:27:02,3,23:27:14,3.0,23:34:00,9.0
373015,449162,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10,23:34:00,10,10,23:27:02,3,23:27:14,3.0,23:34:00,10.0
373016,449163,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,13,23:34:00,10,10,23:27:02,3,23:27:14,3.0,23:34:00,13.0
373017,449164,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10,23:34:00,10,10,23:27:02,3,23:27:14,3.0,23:34:00,10.0


In [14]:
data2

,Unnamed: 0,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,delay,arrival,arrival_wait,delay_last,arrival_-24,delay_-24,arrival_-48,delay_-48,arrival_-week,delay_-week
0,0,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4,06:46:00,4,4,06:46:00,4,06:46:00,4.0,06:46:00,4.0
1,1,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4,07:06:00,4,4,07:06:00,4,07:06:00,4.0,07:06:00,4.0
2,2,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,4,07:26:00,4,4,07:26:00,4,07:26:00,4.0,07:26:00,4.0
3,3,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,4,06:46:00,4,4,06:46:00,4,06:46:00,4.0,06:46:00,4.0
4,4,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,4,07:06:00,4,4,07:06:00,4,07:06:00,4.0,07:06:00,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373014,449161,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,9,23:34:00,10,10,23:27:02,3,23:27:14,3.0,23:34:00,9.0
373015,449162,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10,23:34:00,10,10,23:27:02,3,23:27:14,3.0,23:34:00,10.0
373016,449163,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,13,23:34:00,10,10,23:27:02,3,23:27:14,3.0,23:34:00,13.0
373017,449164,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10,23:34:00,10,10,23:27:02,3,23:27:14,3.0,23:34:00,10.0


In [17]:
list(data2["timetable"])[0].time()

AttributeError: ignored

In [64]:
data

,Unnamed: 0.1,stop_no,query_time,expected_arrival,route_no,destination,gps_tracked,stop_name,date,hour,timetable,day,is_bank_holiday,humidity,rain,temp,sun_hours,wait_time,arrival,arrival_delay,delay_last,wait_time_last,arrival_-24,delay_-24,wait_time_-24,arrival_-48,delay_-48,wait_time_-48,arrival_-week,wait_time_-week,delay_-week
0,10218,214551,06:29:21,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,17,06:46:00,4,17.0,17.0,06:46:00,17,17,06:46:00,17,17,06:46:00,17,17
1,10219,214551,06:29:21,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,37,07:06:00,4,17.0,37.0,07:06:00,37,37,07:06:00,37,37,07:06:00,37,37
2,10220,214551,06:29:21,07:26:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:22:00,4,0,84,0.0,9.6,0.7,57,07:26:00,4,37.0,57.0,07:26:00,57,57,07:26:00,57,57,07:26:00,57,57
3,10221,214551,06:32:25,06:46:00,208,Curraheen,true,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,06:42:00,4,0,84,0.0,9.6,0.7,14,06:46:00,4,17.0,14.0,06:46:00,17,14,06:46:00,17,14,06:46:00,14,17
4,10222,214551,06:32:25,07:06:00,208,Curraheen,false,Bishopstown Road (Wilton Centre),2016-06-17,06:00:00,07:02:00,4,0,84,0.0,9.6,0.7,34,07:06:00,4,17.0,34.0,07:06:00,37,34,07:06:00,37,34,07:06:00,34,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449161,4639073,214551,23:21:26,23:33:33,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,13,23:34:00,10,57.0,13.0,23:27:02,47,6,23:27:14,55,6,23:34:00,13,13
449162,4639074,214551,23:24:29,23:34:32,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,10,23:34:00,10,57.0,10.0,23:27:02,47,3,23:27:14,55,3,23:34:00,10,10
449163,4639075,214551,23:27:32,23:37:21,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,7,23:34:00,10,57.0,7.0,23:27:02,47,0,23:27:14,55,0,23:34:00,7,7
449164,4639076,214551,23:30:34,23:34:52,208,Bishopstown via CUH,true,Bishopstown Road (Wilton Centre),2017-09-24,23:00:00,23:24:00,6,0,93,0.0,11.3,0.0,4,23:34:00,10,57.0,4.0,23:27:02,47,-3,23:27:14,55,-3,23:34:00,4,4


In [0]:
data.to_csv("/content/drive/My Drive/vmware/Project/214551_finalised_dataset.csv")